<div align='center'><font size="10" color="blue"> TEXT GENERATION BY LSTM </font></div>

240108 - Nguyễn Lưu Phương Ngọc Lam 

Kaggle data path: <a> https://www.kaggle.com/datasets/iambestfeeder/10000-vietnamese-books/versions/1 </a>

Aims: Build a LSTM neural network for Text Generation task.
- This code to concat all raw data of 10 Topics newspaper from `txt` file into a `dataframe` and preprocessed by dataset of `hugging face`
- Normalize data and tokenize by PhoBERT Tranformers
- Build model with performance in ...

In [37]:
!pip install datasets transformers -q
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import random
import pickle
from string import punctuation
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

## 1. Load data from kaggle path

In [2]:
file_path = '/kaggle/input/10000-vietnamese-books/output'

# Initialize lists to store file names and their contents
data = []
name = []

# Load files: 137 seconds
for filename in os.listdir(file_path):
    name.append(filename.split('.')[0])  # Extract the filename without extension
    filepath = os.path.join(file_path, filename)
    
    # Open and read the content of the file
    with open(filepath, 'r', encoding='utf-8') as f:
        data.append(f.read())

# Create a DataFrame
df = pd.DataFrame({
    'name': name,
    'content': data
})

In [3]:
# Take a look for few rows of the DataFrame
display(df.sample(5))

,name,content
9276,Anh và em sinh đôi - Phan Hồn Nhiên,\nPhan Hồn Nhiên\nAnh và em sinh đôi\nSinh đôi...
679,Chuyện Của Vi và Tôi - Phạm Nghi Dung,\nPhạm Nghi Dung\nChuyện Của Vi và Tôi\nLớp 5A...
8956,Danuble một dòng còn quyến luyến - Dương Thụy,\nDương Thụy\nDanuble một dòng còn quyến luyến...
10043,"Cô áo hồng, cô áo tím - Lê văn thảo","\nLê văn thảo\nCô áo hồng, cô áo tím\nH\nai ng..."
572,Trong Cơn Mưa - Khúc Thụy Du,\nKhúc Thụy Du\nTrong Cơn Mưa\nLời tác giả\nKí...


## 2. Preprocessing data

In [6]:
from datasets import Dataset

# Chuyển đổi DataFrame thành dict để tạo dataset
df_dict = df.to_dict(orient='list')  
cleaning_dataset = Dataset.from_dict(df_dict)

In [11]:
def extract_author_and_book(examples):
    book_names = []
    authors = []
    
    for name, content in zip(examples['name'], examples['content']):
        # Split the name to get the book name
        parts = name.split(' - ')
        book_name = parts[0].strip()
        
        # Extract the relevant part from content
        relevant_section = content.split('\n')[1:3]  # Get the second and third lines
        author = relevant_section[0].strip()  # Author from the second line
        book_name = relevant_section[1].strip()  # Book name from the third line

        # Append the results
        book_names.append(book_name)
        authors.append(author)
    
    return {'book_name': book_names, 'author': authors}

# Apply the extract_author_and_book function in a batched way - 31 seconds
cleaning_dataset = cleaning_dataset.map(extract_author_and_book, batched=True)

# Convert back to DataFrame
cleaning_df = cleaning_dataset.to_pandas()
cleaning_df = cleaning_df[['book_name', 'author', 'content']]


Map:   0%|          | 0/10415 [00:00<?, ? examples/s]

In [19]:
display(cleaning_df.sample(5))

,book_name,author,content
6146,Ông Thợ Giày Và Cô Con Gái,Cao Hành Kiện,\nCao Hành Kiện\nÔng Thợ Giày Và Cô Con Gái\nT...
3969,Chiếc Vòng Pha Lê,QUỲNH DAO,\nQUỲNH DAO\nChiếc Vòng Pha Lê\nVào lúc năm cù...
108,Bao giờ thì cưới?,Tường Vi,\nTường Vi\nBao giờ thì cưới?\nTom và nàng yêu...
4391,Lụm Còi,Nguyễn Ngọc Tư,"\nNguyễn Ngọc Tư\nLụm Còi\nTôi quyết định rồi,..."
6063,Một Truyện Ngắn Hay Nhất,Linh Bảo,\nLinh Bảo\nMột Truyện Ngắn Hay Nhất\n(trích t...


In [20]:
def normalize_text(text):    
    # Remove content starting from "Mục lục"
    index = text.find("Mục lục")
    if index != -1:
        text = text[:index]
    
    # Remove HTML tags, URLs
    text = re.sub(r'<[^>]*>', '', text)
    text = re.sub(r'Nguồn:\s*http?:\/\/\S+', '', text)
    text = re.sub(r'http\S*', '', text)
    
    # Remove punctuation, digits, and special symbols
    text = re.sub(f'[{punctuation}₫—℅\d\n\t]', ' ', text)
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def clean_content(text):
    results = []
    for book_name, author, content in zip(text['book_name'], text['author'], text['content']):
        # Loại bỏ book_name và author khỏi content
        cleaned_content = re.sub(re.escape(book_name), '', content, flags=re.IGNORECASE)
        cleaned_content = re.sub(re.escape(author), '', cleaned_content, flags=re.IGNORECASE)
        
        # Chuẩn hóa nội dung
        cleaned_content = normalize_text(cleaned_content)
        results.append(cleaned_content)
    return {'cleaned_content': results}

cleaning_dataset = cleaning_dataset.map(clean_content, batched=True) # 5 minutes

# Chuyển đổi thành df để display
cleaning_df = cleaning_dataset.to_pandas()
display(cleaning_df.sample(5))

Map:   0%|          | 0/10415 [00:00<?, ? examples/s]

,name,content,book_name,author,cleaned_content
4659,Ngày tháng nào - Tôn Nữ Thu Dung,\nTôn Nữ Thu Dung\nNgày tháng nào\nChương 1\nE...,Ngày tháng nào,Tôn Nữ Thu Dung,chương em khoan khoái nhìn căn phòng vừa được ...
5297,Đỗ nương nương báo oán - Hồ Biểu Chánh,\nHồ Biểu Chánh\nĐỗ nương nương báo oán\nChươn...,Đỗ nương nương báo oán,Hồ Biểu Chánh,chương bĩ thới tuần huờn trái đất vần xây tối ...
3712,Dòng sông thơ ấu - Hồ Huy Sơn,\nHồ Huy Sơn\nDòng sông thơ ấu\n1. Bố mẹ tôi l...,Dòng sông thơ ấu,Hồ Huy Sơn,bố mẹ tôi ly dị bên nội không ai chịu nhận tôi...
7892,Chim khách kêu - Nguyễn Kiên,\nNguyễn Kiên\nChim khách kêu\nBuổi sáng có co...,Chim khách kêu,Nguyễn Kiên,buổi sáng có con chim khách đến kêu trước cửa ...
8685,Bài toán - Vũ Thư Nguyên,\nVũ Thư Nguyên\nBài toán\nVăn đẩy tấm cửa kiế...,Bài toán,Vũ Thư Nguyên,văn đẩy tấm cửa kiếng lách mình ra ngoài lan c...


In [21]:
cleaning_df = cleaning_df[['book_name', 'author', 'cleaned_content']]
display(cleaning_df.sample(5))

output_file_path = '/kaggle/working/cleaning_data.csv'
cleaning_df.to_csv(output_file_path, index = False)
print(f"File saved to {output_file_path}")

,book_name,author,cleaned_content
2947,Nói không với tiêu cực,Bùi Đức Hiền,sáu giờ chiều trời sẩm tối đèn đã bật sáng tro...
8100,Ven Hồ,Lê Thị Thu Thủy,bây giờ mọi cái không còn đủ sức làm tôi phải ...
7720,Bảy ngày trong đời,Nguyễn Thị Thu Huệ,mai anh đi mấy giờ lụa hỏi bốn giờ sáng bé tí ...
3583,Điện Biên Phủ - Điểm hẹn lịch sử,Võ Nguyên Giáp,chương cuộc họp ở tỉn keo cuộc kháng chiế...
1779,Nữ Chúa Hồ Ba Bể,Hoàng Ly - Đỗ Hồng Linh,phần thứ nhất chương loạn rừng thượng du đất v...


File saved to /kaggle/working/cleaning_data.csv


In [ ]:
# Group by author and aggregate the count of books
book_count = df.groupby('author').size().reset_index(name='number_of_books')

print(book_count)

In [13]:
# Download file from Kaggle
from IPython.display import FileLink
FileLink('cleaning_data.csv')

/kaggle/working/cleaning_data.csv

> The processing of the DataFrame as described above aims to enhance classification tasks if applicable.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

# Tokenize the content data - 26 minutes
def tokenize_function(text):
    return tokenizer(text['cleaned_content'], padding=True, truncation=True, max_length=512)

cleaned_dataset = cleaning_dataset.map(tokenize_function, batched=True)

In [21]:
output_file_path = '/kaggle/working/cleaned_data.csv'
cleaned_df = cleaned_dataset.to_pandas()
cleaned_df.to_csv(output_file_path, index = False)
print(f"File saved to {output_file_path}")

FileLink('cleaned_data.csv') # Download file from Kaggle

File saved to /kaggle/working/cleaned_data.csv


/kaggle/working/cleaned_data.csv

In [28]:
# Calculate text length (number of characters)
cleaned_df['text_length'] = cleaned_df['cleaned_content'].apply(len)
cleaned_df.head()

,name,content,book_name,author,cleaned_content,input_ids,token_type_ids,attention_mask,text_length
0,Nhà ảo thuật - Mạc Can,\nMạc Can\nNhà ảo thuật\nCó một cậu bé muốn họ...,Nhà ảo thuật,Mạc Can,có một cậu bé muốn học vài trò ảo thuật nhưng ...,"[0, 10, 16, 881, 308, 202, 222, 515, 2680, 215...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",9901
1,Ra đi - Trần Chi Liên,\nTrần Chi Liên\nRa đi\nTặng người đồng cảnh t...,Ra đi,Trần Chi Liên,tặng người đồng cảnh tương lân thế rồi những n...,"[0, 806, 18, 80, 805, 8840, 14481, 570, 182, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",12375
2,Đường về - Vũ Thư Nguyên,\nVũ Thư Nguyên\nĐường về\nVừa ra khỏi xa lộ 6...,Đường về,Vũ Thư Nguyên,vừa ra khỏi xa lộ kenneth tăng tốc độ rẽ vào x...,"[0, 164, 40, 353, 604, 1776, 1493, 13564, 2015...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",20383
3,Bốn Thằng Buồn - Phi Va,\nPhi Va\nBốn Thằng Buồn\n&quot;Bốn người lính...,Bốn Thằng Buồn,Phi Va,quot bốn người lính buồn vẫn đó trầm lặng nghĩ...,"[0, 2845, 12440, 1586, 18, 2471, 1520, 74, 37,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3098
4,Chiếc Lexus và cây Ô liu - Thomas L,\nThomas L. Friedman\nChiếc Lexus và cây Ô liu...,Chiếc Lexus và cây Ô liu,Thomas L,friedman lời mở đầu đây là ấn bản bìa mềm cuốn...,"[0, 17869, 3995, 3206, 278, 548, 127, 97, 8, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1120791


In [30]:
columns_to_keep = ['book_name', 'author', 'cleaned_content', 'input_ids', 'token_type_ids', 'attention_mask']
cleaned_dataset = cleaned_dataset.select_columns(columns_to_keep)

In [31]:
cleaned_dataset

Dataset({
    features: ['book_name', 'author', 'cleaned_content', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10415
})

## 3. Split the dataset to train - test set

In [3]:
# Load data if the session crash
path = '/kaggle/input/data-ids/data_ids.xlsx'
df = pd.read_excel(path)
df['input_ids'] = df['input_ids'].apply(lambda x: [int(i) for i in x.strip('[]').split()])
df.head()

,input_ids
0,"[0, 10, 16, 881, 308, 202, 222, 515, 2680, 215..."
1,"[0, 806, 18, 80, 805, 8840, 14481, 570, 182, 2..."
2,"[0, 164, 40, 353, 604, 1776, 1493, 13564, 2015..."
3,"[0, 2845, 12440, 1586, 18, 2471, 1520, 74, 37,..."
4,"[0, 17869, 3995, 3206, 278, 548, 127, 97, 8, 4..."


## 4. Define model

In [30]:
from transformers import AutoTokenizer
from tensorflow.keras.models import load_model  # Import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, Dense

class LSTMModel:
    def __init__(self, vocab_size, embedding_dim, lstm_units, max_length):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.lstm_units = lstm_units
        self.max_length = max_length
        
        # Initialize the model
        self.model = self.build_model()
        
        # Load the tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
        
    def build_model(self):
        """Builds and returns the LSTM model."""
        model = Sequential()
        model.add(Input(shape=(self.max_length,)))  # Add Input Layer with shape
        model.add(Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim))
        model.add(LSTM(self.lstm_units, return_sequences=True))
        model.add(Dropout(0.5))  # Dropout for regularization
        model.add(LSTM(self.lstm_units))
        model.add(Dense(self.vocab_size, activation='softmax'))  # Softmax for generative model
        return model

    def compile_model(self):
        """Compiles the model."""
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    def fit(self, X_train, y_train, epochs=10, batch_size=32, validation_split=0.1):
        """Trains the model on the training data."""
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split, callbacks=[early_stopping])

    def evaluate(self, X_test, y_test):
        """Evaluates the model on the test data."""
        loss, accuracy = self.model.evaluate(X_test, y_test)
        print(f'|Test Loss: {loss}| --------------------- |Test Accuracy: {accuracy}|')

    def save_model(self, filepath):
        """Saves the model to the specified filepath."""
        self.model.save(filepath)
        print(f'Model saved to {filepath}')

    @classmethod
    def load_model(cls, filepath):
        """Loads a model from the specified filepath."""
        model = load_model(filepath)  # Use load_model from keras
        instance = cls(vocab_size=10000, embedding_dim=128, lstm_units=64, max_length=50)  # Use the parameters you defined
        instance.model = model
        instance.tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")  # Reload the tokenizer
        return instance

    def generate_text(self, seed_text, gen_length):
        """Generates text given a seed input and desired length."""
        generated_text = seed_text
        for _ in range(gen_length):
            # Convert seed text to a sequence
            input_sequence = [self.text_to_sequence(generated_text)]
            input_sequence = pad_sequences(input_sequence, maxlen=self.max_length, padding='post')

            # Get the predicted next word
            predicted_probs = self.model.predict(input_sequence, verbose=0)
            predicted_word_index = np.argmax(predicted_probs, axis=-1)[0]

            # Convert the predicted index back to a word
            next_word = self.sequence_to_text(predicted_word_index)

            # Update generated_text with the new word
            generated_text += " " + next_word
        return generated_text

    def text_to_sequence(self, text):
        """Converts text to a sequence of integers based on the vocabulary."""
        return self.tokenizer.encode(text, add_special_tokens=False)

    def sequence_to_text(self, index):
        """Converts an index back to text."""
        return self.tokenizer.decode([index], skip_special_tokens=True)  # Convert index to list for decoding

def prepare_data_from_sequences(input_sequences, total_words):
    """
    Convert input sequences into predictors and labels, similar to a language model task.
    Each sequence will be used to predict the next word in the sequence.
    """
    # Calculate the maximum sequence length
    max_sequence_len = max([len(x) for x in input_sequences])
    
    # Pad sequences to have uniform length
    input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
    
    # Split sequences into predictors and labels
    predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
    
    # Convert labels to categorical (one-hot encoding)
    label = to_categorical(label, num_classes=total_words)
    
    return predictors, label, max_sequence_len

# Usage example
if __name__ == "__main__":
    path = '/kaggle/input/data-ids/data_ids.xlsx'
    df = pd.read_excel(path)
    df['input_ids'] = df['input_ids'].apply(lambda x: [int(i) for i in x.strip('[]').split()])    
    
    # Set the total number of unique tokens based on your tokenizer
    total_words = 10000  # Adjust this as necessary

    # Prepare predictors and labels
    predictors, labels, max_sequence_len = prepare_data_from_sequences(df['input_ids'].tolist(), total_words)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(predictors, labels, test_size=0.2, random_state=42)

    # Set parameters for the LSTM model
    vocab_size = total_words
    embedding_dim = 128
    lstm_units = 64

    # Create an instance of LSTMModel
    lstm_model = LSTMModel(vocab_size, embedding_dim, lstm_units, max_sequence_len)

    # Compile the model
    lstm_model.compile_model()

    # Print model summary
    lstm_model.model.summary()

    # Train the model
    lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

    # Save the model
    lstm_model.save_model("lstm_text_generator.h5")


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 512, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 512, 64)        │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10000)          │       650,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,012,432 (7.68 MB)

 Trainable params: 2,012,432 (7.68 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 14s 51ms/step - accuracy: 0.9555 - loss: 4.0379 - val_accuracy: 0.9868 - val_loss: 0.0889
Epoch 2/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.9755 - loss: 0.1298 - val_accuracy: 0.9868 - val_loss: 0.0801
Epoch 3/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.9793 - loss: 0.1070 - val_accuracy: 0.9868 - val_loss: 0.0779
Epoch 4/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.9767 - loss: 0.1141 - val_accuracy: 0.9868 - val_loss: 0.0741
Epoch 5/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.9768 - loss: 0.1128 - val_accuracy: 0.9868 - val_loss: 0.0728
Epoch 6/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.9763 - loss: 0.1140 - val_accuracy: 0.9868 - val_loss: 0.0886
Epoch 7/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.9756 - loss: 0.1222 - val_accuracy: 0.9868 - val_loss: 0.0679
Epoch 8/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.9785 - loss: 0.0684 - 

## 5. Text generation Inference

In [31]:
# Load the saved model
loaded_model = LSTMModel.load_model("lstm_text_generator.h5")

# Perform inference and generate text based on seed input
seed_text = "Hôm nay trời"  # Example seed text
generated_text = loaded_model.generate_text(seed_text, gen_length=20) 
print("Generated text:", generated_text)

Generated text: Hôm nay trời                    


In [39]:

# Chuẩn bị dữ liệu
# Load data if the session crash
path = '/kaggle/input/data-ids/data_ids.xlsx'
df = pd.read_excel(path)
df['input_ids'] = df['input_ids'].apply(lambda x: [int(i) for i in x.strip('[]').split()])
input_sequences = df['input_ids'].tolist()

# Kiểm tra giá trị tối đa trong input_ids
max_value = max(max(seq) for seq in input_sequences)
print("Giá trị lớn nhất trong input_ids:", max_value)

# Cập nhật total_words dựa trên giá trị tối đa
total_words = max_value + 1  # Thêm 1 để bao gồm giá trị lớn nhất

def prepare_data_from_sequences(input_sequences, total_words):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
    predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
    label = to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

# Gọi hàm để chuẩn bị dữ liệu
X, y, max_sequence_length = prepare_data_from_sequences(input_sequences, total_words)

# Khởi tạo mô hình LSTM
embedding_dim = 128  # Kích thước của vector nhúng

model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Biên dịch mô hình
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X, y, batch_size=64, epochs=10, validation_split=0.2)

# Hàm sinh văn bản
def generate_text(seed_ids, gen_length=100):
    generated_sequence = seed_ids.copy()
    
    for _ in range(gen_length):
        padded_input = pad_sequences([generated_sequence], maxlen=max_sequence_length, padding='pre')
        predicted = model.predict(padded_input, verbose=0)
        predicted_index = np.argmax(predicted, axis=-1)[0]
        generated_sequence.append(predicted_index)

    return generated_sequence

# Sử dụng hàm generate_text để sinh văn bản
seed_text = [0]  # Sử dụng ID ban đầu như là seed
generated_ids = generate_text(seed_text, gen_length=100)

# In ra kết quả
print("Generated IDs:", generated_ids)

Giá trị lớn nhất trong input_ids: 63999
Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 18s 113ms/step - accuracy: 0.9377 - loss: 6.3043 - val_accuracy: 0.9779 - val_loss: 0.1291
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 13s 102ms/step - accuracy: 0.9790 - loss: 0.1195 - val_accuracy: 0.9779 - val_loss: 0.1150
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 14s 103ms/step - accuracy: 0.9820 - loss: 0.0982 - val_accuracy: 0.9779 - val_loss: 0.1111
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 13s 102ms/step - accuracy: 0.9797 - loss: 0.1043 - val_accuracy: 0.9779 - val_loss: 0.1102
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 13s 102ms/step - accuracy: 0.9771 - loss: 0.1125 - val_accuracy: 0.9779 - val_loss: 0.1113
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 13s 102ms/step - accuracy: 0.9792 - loss: 0.1041 - val_accuracy: 0.9779 - val_loss: 0.1080
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 13s 103ms/step - accuracy: 0.9809 - loss: 0.0968 - val_accuracy: 0.9779 - val_loss: 0.1076
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 13s

Kết quả như vậy cho thấy mô hình của tôi chưa học được mối quan hệ ngữ nghĩa trong dữ liệu.

--------------------